In [19]:
import pandas as pd
import numpy as np
import os.path

from keras.preprocessing.image import Iterator
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import load_model

import threading
lock = threading.Lock()

from keras.callbacks import ModelCheckpoint

from BsonIterator import BSONIterator

In [20]:
METADATA_DIR = 'metadata'
DATA_DIR = '../../dataset'

train_offsets_df = pd.read_csv(os.path.join(METADATA_DIR,"train_offsets.csv"), index_col=0)
train_images_df = pd.read_csv(os.path.join(METADATA_DIR,"RandomSplit_Train_0.5_0.1.csv"), index_col=0)
val_images_df = pd.read_csv(os.path.join(METADATA_DIR,"RandomSplit_Val_0.5_0.1.csv"), index_col=0)

num_classes = 5270
num_train_images = len(train_images_df)
num_val_images = len(val_images_df)
batch_size = 64

# Create a generator for training and a generator for validation.
# Tip: use ImageDataGenerator for data augmentation and preprocessing.
assert os.path.exists(os.path.join(DATA_DIR, 'train.bson'))
train_bson_file = open(os.path.join(DATA_DIR, 'train.bson'), "rb")
train_datagen = ImageDataGenerator()
train_gen = BSONIterator(train_bson_file, train_images_df, train_offsets_df,
                         num_classes, train_datagen, lock, target_size = (224,224),
                         batch_size=batch_size, shuffle=True)

val_datagen = ImageDataGenerator()
val_gen = BSONIterator(train_bson_file, val_images_df, train_offsets_df,
                       num_classes, val_datagen, lock, target_size = (224,224),
                       batch_size=batch_size, shuffle=True)

Found 5570369 images belonging to 5270 classes.
Found 614245 images belonging to 5270 classes.


In [21]:
from keras.applications.mobilenet import MobileNet
from keras.preprocessing import image
from keras.models import Model,load_model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.applications import mobilenet

from keras.callbacks import ModelCheckpoint

# Save model Dir
MODEL_DIR = 'saved_models'

# Checkpointer 
checkpointer = ModelCheckpoint(filepath=MODEL_DIR+'/Mobilenet_finetune.{epoch:02d}-{acc:.2f}-{val_acc:.2f}_RandomSplit_Train_0.5_0.1.h5', verbose=1)

# create the base pre-trained model
model = load_model(MODEL_DIR+'/Mobilenet.Epochs10_Acc0.58_Val0.52_RandomSplit_Train_0.9_0.1.h5', \
                  custom_objects={'relu6': mobilenet.relu6,'DepthwiseConv2D': mobilenet.DepthwiseConv2D})

for layer in model.layers[:64]:
   layer.trainable = False
for layer in model.layers[64:]:
   layer.trainable = True


In [ ]:
# compile the model (should be done *after* setting layers to non-trainable)
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
num_epochs = 5


# train the model on the new data for a few epochs
model.fit_generator(train_gen,
                    steps_per_epoch = num_train_images // batch_size,
                    epochs = num_epochs,
                    validation_data = val_gen,
                    validation_steps = num_val_images // batch_size,
                    workers = 4, callbacks=[checkpointer])

Epoch 1/5
87037/87037 [==============================] - ETA: 0s - loss: 2.6858 - acc: 0.5169 - val_loss: 2.5510 - val_acc: 0.5308
Epoch 2/5
 7508/87037 [=>............................] - ETA: 14226s - loss: 2.5190 - acc: 0.5316